In [ ]:
#!conda install gdcm -c conda-forge -y

In [ ]:
! pip install tf-madgrad

In [ ]:
! pip install wandb

In [ ]:
!pip install efficientnet -q

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf 
import os
from sklearn.model_selection import GroupKFold
from kaggle_datasets import KaggleDatasets
from glob import glob
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
from tensorflow.keras import Model, Sequential
from tqdm import tqdm
from PIL import Image
import cv2 as cv
import wandb
from wandb.keras import WandbCallback
import gc
import tensorflow.keras.backend as K
import efficientnet.tfkeras as efn
from madgrad import MadGrad

In [ ]:
key='12db35704d46871af0f2b6ee96b113f5ed91b5fd'
wandb.login(key=key)

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU: {}'.format(tpu.master()))
except ValueError:
    tpu = None
    
if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
def decode_image(path, label=None, image_size=(512, 512), mask=False):
    image = tf.io.read_file(path)
    image = tf.image.decode_png(image, channels=1) if mask else tf.image.decode_png(image, channels=3)
    image = tf.cast(image, tf.float32) / 255.
    image = tf.image.resize(image, (16, 16)) if mask else tf.image.resize(image, image_size)
    if label is not None:
        return image, label
    return image

In [ ]:
'''def apply_augmentation(image, mask, label):
    image = tf.image.random_flip_up_down(image)
    #mask = tf.image.random_flip_up_down(mask)
    image = tf.image.random_flip_left_right(image)
    #mask = tf.image.random_flip_left_right(mask)
    if label is not None:
        return image, mask, label
    return image, mask, label'''

In [ ]:
def left_right_flip(image, mask, label):
    image = tf.image.flip_left_right(image)
    mask = tf.image.flip_left_right(mask)
    return image, mask, label

In [ ]:
def up_down_flip(image, mask, label):
    image = tf.image.flip_up_down(image)
    mask = tf.image.flip_up_down(mask)
    return image, mask, label

In [ ]:
def rotate_image(image, mask, label):
    k = np.random.choice([1, 2, 3])
    image = tf.image.rot90(image, k=k)
    mask = tf.image.rot90(mask, k=k)
    return image, mask, label

In [ ]:
def apply_augmentation(image, mask, label):
    fn = np.random.choice([lambda image, mask, label: left_right_flip(image, mask, label),
                           lambda image, mask, label: up_down_flip(image, mask, label),
                           lambda image, mask, label: rotate_image(image, mask, label)])
    image, mask, label = fn(image, mask, label)
    return image, mask, label

In [ ]:
BATCH_SIZE = strategy.num_replicas_in_sync * 16 if tpu else 32
AUTO = tf.data.experimental.AUTOTUNE

In [ ]:
class MultiTask(Model):
    def __init__(self):
        super(MultiTask, self).__init__()
        self.base_model = self.create_base((512, 512, 3))
        self.mask = self.create_mask()
        self.classification = self.create_classification()
        
    @staticmethod   
    def create_base(input_shape):
        #inputs = tf.keras.layers.Input(shape=input_shape)
        #base_model = tf.keras.applications.EfficientNetB7(weights='imagenet', 
                                                          #include_top=False, 
                                                          #input_shape=input_shape)
        #base = base_model(inputs)
        base_model = efn.EfficientNetB3(input_shape=input_shape, weights='imagenet', include_top=False)
        model = Model(inputs=base_model.inputs, outputs=[base_model.get_layer('top_activation').output, base_model.output])
        return model
    
    @staticmethod
    def create_mask():
        mask_model = Sequential([
            tf.keras.layers.Conv2D(filters=512, kernel_size=(1, 1), strides=(1, 1), padding="same"),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.ReLU(),
            tf.keras.layers.Conv2D(filters=1, kernel_size=(1,1), padding="same")
        ], name='mask')

        return mask_model
    
    @staticmethod
    def create_classification():
        classification_model = Sequential([
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dense(4, activation='softmax')                                             
        ], name='target')
        
        return classification_model
    
    def call(self, inputs, **kwargs):
        segmentation_part, classification_part = self.base_model(inputs['input'])
        mask = self.mask(segmentation_part)
        classification = self.classification(classification_part)
        return {
            'clss' : classification,
            'segm' : mask
        }
    
        

In [ ]:
tf.keras.backend.clear_session()
model = MultiTask()
model.build(input_shape={'input': (None, 512, 512, 3)})
model.summary()
#tf.keras.utils.plot_model(model)

In [ ]:
df = pd.read_csv('../input/siim-covid19-detection/train_study_level.csv')
groupkfold = GroupKFold(n_splits=5)
df['fold'] = -1
for fold, (train_indx, valid_indx) in enumerate(groupkfold.split(df, groups=df.id.tolist())):
    df.loc[valid_indx, 'fold'] = fold

In [ ]:
def multitask_processing(image_path, mask_path, label):
    #im_path = '../input/covid19-detection-890pxpng-study/train/' + image_path
    #mask_path = '../input/covid19-detection-890pxpng-study/ROI Mask/' + mask_path
    image = decode_image(image_path)
    mask = decode_image(mask_path, mask=True)
    return image, mask, label

In [ ]:
#@tf.function
def create_dataset(image_paths, mask_paths, labels, train=True):
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, mask_paths, labels))
    dataset = dataset.map(multitask_processing, num_parallel_calls=AUTO)
    dataset = dataset.map(apply_augmentation, num_parallel_calls=AUTO) if train else dataset
    #dataset = dataset.shuffle(2048)
    dataset = dataset.map(lambda image, mask, label: ({'input': image}, {'clss': label, 'segm': mask}))
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    #dataset = dataset.map(lambda image, mask, label: {'input': image}, {'clss': label, 'segg': mask})
    return dataset

In [ ]:
COMPETITION_NAME = "covid19-detection-890pxpng-study"
GCS_DS_PATH = KaggleDatasets().get_gcs_path(COMPETITION_NAME)

In [ ]:
mask_paths = os.listdir('../input/covid19-detection-890pxpng-study/ROI Mask/')
image_paths = os.listdir('../input/covid19-detection-890pxpng-study/train/')

In [ ]:
def check_path(study_id):
    if study_id + '.png' in mask_paths:
        #path = '../input/covid19-detection-890pxpng-study/ROI Mask/' + study_id + '.png'
        path = GCS_DS_PATH + '/ROI Mask/' + study_id + '.png'
    else:
        #path = '../input/covid19-detection-890pxpng-study/train/' + study_id + '.png'
        path = GCS_DS_PATH + '/train/' + study_id + '.png'
    return path

In [ ]:
df = pd.read_csv('../input/covid19-detection-890pxpng-study/train.csv')
df['image_path'] = df.id.apply(lambda x: GCS_DS_PATH + '/train/' + x + '.png')
df['mask_path'] = df.id.apply(check_path)

In [ ]:
groupkfold = GroupKFold(n_splits=5)
df['fold'] = -1
for fold, (train_indx, valid_indx) in enumerate(groupkfold.split(df, groups=df.id.tolist())):
    df.loc[valid_indx, 'fold'] = fold

In [ ]:
df.head()

In [ ]:
label_cols = df.columns[4:8]

In [ ]:
class LRWarmup(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, init_lr, total_steps, minimal_lr, warmup_epochs, steps_per_epoch):
        super(LRWarmup, self).__init__()
        self.init_lr = init_lr
        self.total_steps = total_steps
        self.minimal_lr = minimal_lr
        self.warmup_epochs = warmup_epochs
        self.steps_per_epoch = steps_per_epoch
        
    def __call__(self, step):
        lr = 0.5 * self.init_lr * (
              1 + tf.cos(np.pi * tf.cast(step, tf.float32) / self.total_steps))
        
        if self.minimal_lr:
            lr = tf.math.maximum(lr, self.minimal_lr)

        if self.warmup_epochs:
            warmup_steps = int(self.warmup_epochs * self.steps_per_epoch)
            warmup_lr = (
              self.init_lr * tf.cast(step, tf.float32) /
              tf.cast(warmup_steps, tf.float32))
            lr = tf.cond(step < warmup_steps, lambda: warmup_lr, lambda: lr)
        return lr

In [ ]:
for i in range(5):
    valid_image_paths = df[df['fold'] == i]['image_path']
    valid_mask_paths = df[df['fold'] == i]['mask_path']
    train_image_paths = df[df['fold'] != i]['image_path']
    train_mask_paths = df[df['fold'] != i]['mask_path']
    valid_labels = df[df['fold'] == i][label_cols].values
    train_labels = df[df['fold'] != i][label_cols].values
    
    train_dataset = create_dataset(train_image_paths, train_mask_paths, train_labels)
    valid_dataset = create_dataset(valid_image_paths, valid_mask_paths, valid_labels, train=False)
        
    steps_per_epoch = int(train_labels.shape[0]/BATCH_SIZE)
    total_steps = 20*steps_per_epoch
    lr_base = 0.016
    scaled_lr = lr_base * (BATCH_SIZE / 256.0)
    
    
    learning_rate = LRWarmup(init_lr=scaled_lr, 
                             total_steps=total_steps, 
                             minimal_lr=0, 
                             warmup_epochs=5,
                             steps_per_epoch=steps_per_epoch) 
        
    with strategy.scope():
        model = MultiTask()
        #optimizer = tf.keras.optimizers.Adam(learning_rate)
        otimizer = MadGrad(lr=0.001, momentum=0.9, epsilon=1e-06)
        model.compile(optimizer=otimizer, loss={'clss': tf.keras.losses.CategoricalCrossentropy(
                                label_smoothing=0, from_logits=False),
                            'segm': tf.keras.losses.BinaryCrossentropy(from_logits=True)},

                        metrics = {
                            'clss': [
                                tf.keras.metrics.AUC(curve='ROC', multi_label=True),
                                #metrics.SpecificityAtSensitivity(0.60, name='@sensitivity')
                            ]})
        
    run = wandb.init(project='madgrad', 
                 tags=['train', 'kfold', 'mixup', 'effnetb7'],
                 notes='training effnetb7 with aux loss')
        
    model_checkpoint = tf.keras.callbacks.ModelCheckpoint(f'auxloss_model{i}.h5', save_best=True, monitor='val_loss')
    #lr_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=3, min_lr=1e-6, mode='min')
    #print(f'Fold # {i}')
    history = model.fit(train_dataset, 
                        epochs=20,
                        callbacks=[model_checkpoint, WandbCallback()],
                        validation_data = valid_dataset)
    
    
    #loss, auc = model.evaluate(valid_dataset)
    #wandb.log({'Val Loss': loss, 'Val AUC-ROC': auc})
    run.finish()
    
    del model, train_dataset, valid_dataset
    gc.collect()
    K.clear_session()
    break